# Latihan Webscraping - Bhinneka.com
---
Kamis, 26 Juni 2024

## Import Libraries

In [8]:
# Libraries for data manipulation
import pandas as pd

# Libraries for webscraping
from selenium import webdriver
from bs4 import BeautifulSoup


In [11]:
# assign driver
driver = webdriver.Chrome()

In [12]:
# access website
driver.get('https://www.bhinneka.com/jual?page=1&cari=laptop&order=')

In [13]:
# get element
html = driver.page_source

# parse the html
soup = BeautifulSoup(html, 'html.parser')

## Element Exploration

### Product Name

```html
<a class="text-primary text-decoration-none" itemprop="name" href="/sitstand-laptop-stand-alumunium-premium-portable-lap05-sku3340917845" content="SitStand Laptop Stand Alumunium Premium Portable LAP05">SitStand Laptop Stand Alumunium Premium Portable LAP05</a>```

In [16]:
product_name = soup.find('a', {'itemprop':'name'})

product_name.get_text()

'ORICO Notebook Laptop Holder Bracket LST-4A'

### Normal Price

In [30]:
price = soup.find("span", {"class":"bmd-special-price"})

price.get_text().strip()

'Rp. 890.000'

In [22]:
soup.find("span",{"class":"oe_currency_value"}).get_text()

'890.000'

### Installment Price

In [25]:
installment_price = soup.find("span", {"class":"d-flex flex-column"})
installment_price

In [26]:
soup.find("span",{"class":"bmd-installment"}).get_text().strip()

'Cicilan Rp. 70.750/bln'

### Reviewer

In [29]:
soup.find("div", {"class":"bmd-product-rating"}).get_text().strip()

'(0)'

## Webscraping Dataframe

**Trial 1**

In [78]:
# url
url = 'https://www.bhinneka.com/jual?page=1&cari=laptop&order='

# access website
driver.get(url)

# fetch html element
html = driver.page_source

# parsing
page = BeautifulSoup(html, 'html.parser')

# create varibles to keep the value
products = []
prices = []
installments = []
reviewers = []
stores = []

# search for boxes
boxes = page.find_all("div", {"class": "o_wsale_product_information position-relative d-flex flex-column flex-grow-1 flex-shrink-1"})

# get all the variable from each boxes
for box in boxes:
    # find all product
    list_products = box.find('a', {'itemprop':'name'})
    # add to empty list
    for product in list_products:
        products.append(product.get_text().strip())

    # find all price
    list_prices = box.find("span",{"class":"oe_currency_value"})
    # add to empty list
    for price in list_prices:
        prices.append(price.get_text().strip())

    # find all installment
    list_installments = box.find("span",{"class":"bmd-installment"})
    # add to empty list
    for installment in list_installments:
        installments.append(installment.get_text().strip())

    # find all reviewer
    list_reviewers = box.find_all("span", {"class":"bmd-installment"})
    reviewers.append(list_reviewers[2].get_text().strip())

    # find all store
    list_stores = box.find_all("span", {"class":"bmd-installment"})
    stores.append(list_reviewers[1].get_text().strip())



In [81]:
data = pd.DataFrame({
    'nama produk' : products,
    'harga normal': prices,
    'harga cicilan' : installments,
    'jumlah review': reviewers,
    'asal toko' : stores
})

data.head()

,nama produk,harga normal,harga cicilan,jumlah review,asal toko
0,ORICO Notebook Laptop Holder Bracket LST-4A,890.000,Cicilan Rp. 70.750/bln,(0),Jakarta Utara
1,DELL Business Notebook Vostro 3405 (Ryzen 5 35...,9.213.000,Cicilan Rp. 654.167/bln,(2),Jakarta Utara
2,ASUS Notebook M415DAO-FHD321 Slate Grey,5.499.000,Cicilan Rp. 458.250/bln,(0),Jakarta Utara
3,SitStand Laptop Stand Alumunium Premium Portab...,302.000,Cicilan Rp. 24.917/bln,(0),Jakarta Utara
4,ORICO Adjustable Laptop Stand Metal PFB-A2 Silver,149.000,Cicilan Rp. 12.417/bln,(0),Jakarta Utara


**Trial Multipage**

In [86]:
import time

In [87]:
# create varibles to keep the value
products = []
prices = []
installments = []
reviewers = []
stores = []

# looping for several pages
for page in range(1, 6):

    # url
    url = f'https://www.bhinneka.com/jual?page={page}&cari=laptop&order='

    # access website
    driver.get(url)

    # delay
    time.sleep(2)

    # fetch html element
    html = driver.page_source

    # parsing
    page = BeautifulSoup(html, 'html.parser')

    # search for boxes
    boxes = page.find_all("div", {"class": "o_wsale_product_information position-relative d-flex flex-column flex-grow-1 flex-shrink-1"})

    # get all the variable from each boxes
    for box in boxes:

        # trying to get product name 
        try:
            # find all product
            list_products = box.find('a', {'itemprop':'name'})
            # add to empty list
            for product in list_products:
                products.append(product.get_text().strip())
        except:
            products.append(None)

        # find all price
        list_prices = box.find("span",{"class":"oe_currency_value"})
        # add to empty list
        for price in list_prices:
            prices.append(price.get_text().strip())

        # find all installment
        list_installments = box.find("span",{"class":"bmd-installment"})
        # add to empty list
        for installment in list_installments:
            installments.append(installment.get_text().strip())

        # find all reviewer
        list_reviewers = box.find_all("span", {"class":"bmd-installment"})
        reviewers.append(list_reviewers[2].get_text().strip())

        # find all store
        list_stores = box.find_all("span", {"class":"bmd-installment"})
        stores.append(list_stores[1].get_text().strip())




In [88]:
data = pd.DataFrame({
    'nama produk' : products,
    'harga normal': prices,
    'harga cicilan' : installments,
    'jumlah review': reviewers,
    'asal toko' : stores
})

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nama produk    200 non-null    object
 1   harga normal   200 non-null    object
 2   harga cicilan  200 non-null    object
 3   jumlah review  200 non-null    object
 4   asal toko      200 non-null    object
dtypes: object(5)
memory usage: 7.9+ KB


**Final Code**

In [93]:
# create varibles to keep the value
products = []
prices = []
installments = []
reviewers = []
stores = []

# looping for several pages
for page in range(1, 3):

    # url
    url = f'https://www.bhinneka.com/jual?page={page}&cari=laptop&order='

    # access website
    driver.get(url)

    # delay
    time.sleep(2)

    # fetch html element
    html = driver.page_source

    # parsing
    page = BeautifulSoup(html, 'html.parser')

    # search for boxes
    boxes = page.find_all("div", {"class": "o_wsale_product_information position-relative d-flex flex-column flex-grow-1 flex-shrink-1"})

    # get all the variable from each boxes
    for box in boxes:

        # trying to get product name 
        try:
            # find all product
            product = box.find('a', {'itemprop':'name'})
            # add to empty list
            products.append(product.get_text().strip())
        except:
            products.append(None)

        # trying to get the price
        try:
            # find all price
            price = box.find("span",{"class":"oe_currency_value"})
            # add to empty list
            prices.append(price.get_text().strip())
        except:
            prices.append(None)

        # trying to get the installment price
        try:
            # find all installment
            installment = box.find("span",{"class":"bmd-installment"})
            # add to empty list
            installments.append(installment.get_text().strip())
        except:
            installments.append(None)

        # trying to get the reviewer
        try:
            # find all reviewer
            list_reviewers = box.find_all("span", {"class":"bmd-installment"})
            reviewers.append(list_reviewers[2].get_text().strip())
        except:
            reviewers.append(None)

        # trying to get the store
        try:
            # find all store
            list_stores = box.find_all("span", {"class":"bmd-installment"})
            stores.append(list_stores[1].get_text().strip())
        except:
            stores.append(None)

# creating a dataframe
data = pd.DataFrame({
    'nama produk' : products,
    'harga normal': prices,
    'harga cicilan' : installments,
    'jumlah review': reviewers,
    'asal toko' : stores
})


In [92]:
data.head()

,nama produk,harga normal,harga cicilan,jumlah review,asal toko
0,ORICO Notebook Laptop Holder Bracket LST-4A,890.000,Cicilan Rp. 70.750/bln,(0),Jakarta Utara
1,SitStand Laptop Stand Alumunium Premium Portab...,302.000,Cicilan Rp. 24.917/bln,(0),Jakarta Utara
2,DELL Business Notebook Vostro 3405 (Ryzen 5 35...,9.213.000,Cicilan Rp. 654.167/bln,(2),Jakarta Utara
3,ASUS Notebook M415DAO-FHD321 Slate Grey,5.499.000,Cicilan Rp. 458.250/bln,(0),Jakarta Utara
4,ORICO Adjustable Laptop Stand Metal PFB-A2 Silver,149.000,Cicilan Rp. 12.417/bln,(0),Jakarta Utara
